# Cosmology project: Measuring Cosmic acceleration with Type Ia Supernovaes

In [55]:
import pandas as pd
import matplotlib as plt
import numpy as np

In [75]:
database_file = 'SCPUnion2.1_AllSNe.tex'
SNe_df = pd.read_csv(database_file,header=None, sep=r'\)?\s+&\s+', engine='python', na_filter = ['\nodata','\nodata\\'],
                    names=['ID', 'z', 'Mb',  'stretch', 'color',
                           'd_mod', 'unknown', 'sample', 'Cuts Failed']).set_index('ID')
SNe_df['Cuts Failed'] = SNe_df['Cuts Failed'].str.slice(stop=-2)
SNe_df.head(10)

,z,Mb,stretch,color,d_mod,unknown,sample,Cuts Failed
ID,,,,,,,,
1993ah,0.0285,16.86(0.19,-2.26(0.93,0.23(0.09,35.35(0.22,0.128,1.0,\nodata
1993ag,0.0500,17.79(0.05,-1.09(0.24,0.12(0.02,36.68(0.17,0.128,1.0,\nodata
1993o,0.0529,17.60(0.05,-1.03(0.14,-0.01(0.02,36.82(0.16,0.128,1.0,\nodata
1993b,0.0701,18.43(0.04,-0.53(0.21,0.09(0.02,37.45(0.16,0.128,1.0,\nodata
1992bs,0.0627,18.25(0.05,-0.27(0.23,0.02(0.02,37.48(0.16,0.128,1.0,\nodata
1992br,0.0876,19.19(0.11,-2.97(0.38,-0.04(0.07,38.23(0.19,0.128,1.0,\nodata
1992bp,0.0786,18.27(0.04,-1.27(0.20,-0.02(0.02,37.49(0.16,0.128,1.0,\nodata
1992bo,0.0172,15.75(0.13,-2.68(0.18,0.03(0.02,34.65(0.20,0.128,1.0,\nodata
1992bl,0.0422,17.30(0.08,-1.95(0.24,0.02(0.04,36.34(0.17,0.128,1.0,\nodata


In [76]:
def split_errors(df,name):
    #print df.loc[df[name]==na][name]
    mask = df[name].str.slice(start=-6)=='nodata'
    df.loc[mask, name] = 'NaN(NaN'
    split_df = df[name].str.split('(',expand=True)
    df[name] = split_df[0].astype(float)
    df['err_{}'.format(name)] = split_df[1].astype(float)
split_errors(SNe_df,'Mb')
split_errors(SNe_df,'stretch')
split_errors(SNe_df,'color')
split_errors(SNe_df,'d_mod')
nice_columns = ['z', 'Mb', 'err_Mb',  'color', 'err_color',
                           'd_mod', 'err_d_mod']
SNe_df = SNe_df[nice_columns]
SNe_df.head()

,z,Mb,err_Mb,color,err_color,d_mod,err_d_mod
ID,,,,,,,
1993ah,0.0285,16.86,0.19,0.23,0.09,35.35,0.22
1993ag,0.0500,17.79,0.05,0.12,0.02,36.68,0.17
1993o,0.0529,17.60,0.05,-0.01,0.02,36.82,0.16
1993b,0.0701,18.43,0.04,0.09,0.02,37.45,0.16
1992bs,0.0627,18.25,0.05,0.02,0.02,37.48,0.16


nan <type 'numpy.float64'>
